In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm
tqdm.pandas()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Bidirectional,Convolution1D,Dense,GlobalMaxPool1D,Bidirectional,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from gensim.models import KeyedVectors

In [0]:
df = pd.read_csv('/content/drive/My Drive/EXL Data/Company response reduced 50k.csv')
df.head()

,Consumer complaint narrative,Total Clean Text,After Stop Word Removal,Company response to consumer,Complaint ID
0,"When I received the initial call, I asked the ...",when receive the initial call ask the agent ...,receive initial call ask agent stop call debt ...,Closed with non-monetary relief,2714037
1,Capital One applied my {$100.00} XXXX 2016 pay...,capital one apply my 100 00 2016 payment for ...,capital one apply 100 00 2016 payment card end...,Closed with monetary relief,1886231
2,THIS COMPANY IS REPORTING AN ACCOUNT ON MY CRE...,this company be report an account on my credit...,company report account credit mine inaccurate ...,Closed with non-monetary relief,3448072
3,So I recieved an email from my employer that m...,so recieved an email from my employer that my...,recieved email employer wag garnish state woul...,Closed with non-monetary relief,3351230
4,"XX/XX/XXXX, I started working with XXXX XXXX X...",start work with and his assistant ...,start work assistant regard streamline fh refi...,Others,3352054


In [0]:
df['Company response to consumer'].value_counts()

Closed with non-monetary relief    56652
Others                             50000
Closed with monetary relief        26882
Name: Company response to consumer, dtype: int64

In [0]:
def renaming(text):
  if text == 'Closed with explanation':
    return 'Others'
  elif text == 'Closed':
    return 'Others'
  elif text == 'Untimely response':
    return 'Others'
  else:
    return text

In [0]:
df['Company response to consumer'] = df['Company response to consumer'].progress_apply(renaming)

100%|██████████| 133534/133534 [00:00<00:00, 901479.80it/s]


In [0]:
#df.sort_values(by = 'Company response to consumer',ascending=True,inplace=True)

In [0]:
#df.head()

In [0]:
df['Company response to consumer'].value_counts()

Closed with non-monetary relief    56652
Others                             50000
Closed with monetary relief        26882
Name: Company response to consumer, dtype: int64

In [0]:
#df = df[:130000]
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,Consumer complaint narrative,Total Clean Text,After Stop Word Removal,Company response to consumer,Complaint ID
0,My wife and I obtained a vehicle loan through ...,my wife and obtain vehicle loan through fift...,wife obtain vehicle loan fifth third bank 2016...,Closed with non-monetary relief,2704428
1,"To whom it may concern, XXXX negative accounts...",to whom it may concern negative account ch...,may concern negative account charge account mi...,Closed with non-monetary relief,1652669
2,"Apparently, I made a simple error in my bill p...",apparently make simple error in my bill pay ...,apparently make simple error bill pay bank ame...,Closed with monetary relief,2470412
3,I activated and registered a prepaid XXXX card...,activate and register prepay card use it f...,activate register prepay card use fully within...,Others,3331201
4,On XXXX XXXX XXXX i went into XXXX on to use m...,on go into on to use my atm withdraw th...,go use atm withdraw come crook reach grab side...,Closed with monetary relief,1697770


In [0]:
x = df['Total Clean Text']
y = df['Company response to consumer']

le = LabelEncoder()

y = le.fit_transform(y)
y = to_categorical(y)

In [0]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(x)

seq = tokenizer.texts_to_sequences(x)

pad_seq = pad_sequences(seq,maxlen = 500)

In [0]:
vocab_size = len(tokenizer.word_index)+1
vocab_size

26056

In [0]:
embedding_values = {}
f = open('/content/drive/My Drive/Embeddings/glove.840B.300d.txt')
for line in tqdm(f):
  value = line.split(' ')
  word = value[0]
  coef = np.array(value[1:],dtype = 'float')
  if coef is not None:
    embedding_values[word] = coef

2196018it [03:33, 10276.65it/s]


In [0]:
word2vec = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Embeddings/GoogleNews-vectors-negative300.bin', \
        binary=True)
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Found 3000000 word vectors of word2vec


In [0]:
all_embds = np.stack(embedding_values.values())
mean,std = all_embds.mean(),all_embds.std()
mean,std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(-0.005838497258278652, 0.4878208104560991)

In [0]:
embedding_matrix = np.random.normal(mean,std,(vocab_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
  value = embedding_values.get(word)
  if value is not None:
    embedding_matrix[i] = value

100%|██████████| 26055/26055 [00:00<00:00, 343755.40it/s]


In [0]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)

In [0]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(Embedding(vocab_size,300,input_length=500,weights = [embedding_matrix],trainable = False))
model.add(Bidirectional(GRU(32,return_sequences=True)))
model.add(Convolution1D(64,2,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(3,activation = 'softmax'))

model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(pad_seq,y,test_size = 0.15,random_state= 42,stratify = y)

In [0]:
model.fit(x_train,y_train,batch_size = 32,epochs = 5,validation_split = 0.15 )

Epoch 1/5
3015/3015 [==============================] - 517s 171ms/step - loss: 0.7844 - accuracy: 0.6094 - val_loss: 0.7303 - val_accuracy: 0.6461
Epoch 2/5
3015/3015 [==============================] - 515s 171ms/step - loss: 0.7137 - accuracy: 0.6589 - val_loss: 0.7115 - val_accuracy: 0.6650
Epoch 3/5
3015/3015 [==============================] - 515s 171ms/step - loss: 0.6849 - accuracy: 0.6762 - val_loss: 0.7020 - val_accuracy: 0.6695
Epoch 4/5
3015/3015 [==============================] - 515s 171ms/step - loss: 0.6615 - accuracy: 0.6910 - val_loss: 0.6873 - val_accuracy: 0.6749
Epoch 5/5
3015/3015 [==============================] - 515s 171ms/step - loss: 0.6399 - accuracy: 0.7027 - val_loss: 0.6914 - val_accuracy: 0.6766


In [0]:
loss,acc = model.evaluate(x_test,y_test)
loss,acc

626/626 [==============================] - 43s 69ms/step - loss: 0.6845 - accuracy: 0.6788


(0.6844510436058044, 0.6788477897644043)

In [0]:
model.save('Consumer response Word2Vec 500 Length Reduced.h5')

In [0]:
import pickle

# saving
with open('tokenizer_Consumer_response_Word2Vec_500_Length Reduced.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
le.classes_

array(['Closed with monetary relief', 'Closed with non-monetary relief',
       'Others'], dtype=object)

In [0]:
from sklearn import metrics


predictions = model.predict(x_test)
for thresh in np.arange(0.5, 0.701, 0.01):
    thresh = np.round(thresh, 2)
    print("Precision score at threshold {0} is {1}".format(thresh, metrics.precision_score(y_test, (predictions>thresh).astype(int),average="micro")))

Precision score at threshold 0.5 is 0.704696499778467
Precision score at threshold 0.51 is 0.7100267913127744
Precision score at threshold 0.52 is 0.7163391023760634
Precision score at threshold 0.53 is 0.7225981873111782
Precision score at threshold 0.54 is 0.7302380803599325
Precision score at threshold 0.55 is 0.7347631241997439
Precision score at threshold 0.56 is 0.7404967389156071
Precision score at threshold 0.57 is 0.7466693855356172
Precision score at threshold 0.58 is 0.7534361851332398
Precision score at threshold 0.59 is 0.7606701812667004
Precision score at threshold 0.6 is 0.7673862702621946
Precision score at threshold 0.61 is 0.772972556275054
Precision score at threshold 0.62 is 0.7770076481835564
Precision score at threshold 0.63 is 0.7830235236058563
Precision score at threshold 0.64 is 0.7895407080397318
Precision score at threshold 0.65 is 0.7940507436570429
Precision score at threshold 0.66 is 0.7992029707454035
Precision score at threshold 0.67 is 0.8026733968966

In [0]:
for thresh in np.arange(0.5, 0.701, 0.01):
    thresh = np.round(thresh, 2)
    print("Recall score at threshold {0} is {1}".format(thresh, metrics.recall_score(y_test, (predictions>thresh).astype(int),average="micro")))

Recall score at threshold 0.5 is 0.6352154161050372
Recall score at threshold 0.51 is 0.6218361539613599
Recall score at threshold 0.52 is 0.6095551894563427
Recall score at threshold 0.53 is 0.59702461185163
Recall score at threshold 0.54 is 0.5833957366082572
Recall score at threshold 0.55 is 0.5729619090409864
Recall score at threshold 0.56 is 0.5611302481154211
Recall score at threshold 0.57 is 0.548399980030952
Recall score at threshold 0.58 is 0.5363686286256303
Recall score at threshold 0.59 is 0.5258349558184814
Recall score at threshold 0.6 is 0.5128550746343168
Recall score at threshold 0.61 is 0.5005741101292995
Recall score at threshold 0.62 is 0.4868953122659877
Recall score at threshold 0.63 is 0.4752633418201787
Recall score at threshold 0.64 is 0.464280365433578
Recall score at threshold 0.65 is 0.4530976985672208
Recall score at threshold 0.66 is 0.440517198342569
Recall score at threshold 0.67 is 0.42868553741700366
Recall score at threshold 0.68 is 0.4168039538714991

In [0]:
for thresh in np.arange(0.5, 0.701, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(y_test, (predictions>thresh).astype(int),average="micro")))

F1 score at threshold 0.5 is 0.6681544884081182
F1 score at threshold 0.51 is 0.6630116569968596
F1 score at threshold 0.52 is 0.6586471032473837
F1 score at threshold 0.53 is 0.6538366911784806
F1 score at threshold 0.54 is 0.6486096464450241
F1 score at threshold 0.55 is 0.6438529073518275
F1 score at threshold 0.56 is 0.6384549843794377
F1 score at threshold 0.57 is 0.6323575972138272
F1 score at threshold 0.58 is 0.6266367268379458
F1 score at threshold 0.59 is 0.6218194698624475
F1 score at threshold 0.6 is 0.6148183613621401
F1 score at threshold 0.61 is 0.6076417295397388
F1 score at threshold 0.62 is 0.5986557407236902
F1 score at threshold 0.63 is 0.5915064152350182
F1 score at threshold 0.64 is 0.5847217856020119
F1 score at threshold 0.65 is 0.5769683099710754
F1 score at threshold 0.66 is 0.5679711637487127
F1 score at threshold 0.67 is 0.5588857431091151
F1 score at threshold 0.68 is 0.5502900079093066
F1 score at threshold 0.69 is 0.5414360957397161
F1 score at threshold 